<a href="https://colab.research.google.com/github/PollyBecker/NLP/blob/main/NER_Reconhecimento_de_Entidades_Nomeadas_%7C_spaCy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NER Named Entity Recognition
Named Entity Recognition (NER), em Português, Reconhecimento de Entidade Nomeada, é um processo de identificação de entidades predefinidas presentes em um texto, como nome da pessoa, organização, local que já vem pré-definidas no framework spaCy ou entidades que podemos criar com ajuda desta ferramenta.

# 1- Instalando o SpaCy

https://www.youtube.com/watch?v=2XUhKpH0p4M

https://www.youtube.com/watch?v=9mXoGxAn6pM
https://spacy.io/models

In [ ]:
!pip install -U spacy

# 2- Fazendo o dowload do modelo


In [ ]:
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 22.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
import pt_core_news_sm
nlp = pt_core_news_sm.load()

In [ ]:
texto = nlp('Maria mora em Paris')

# Printando as entidades padão do modelo
for entidade in texto.ents:
  print(entidade.text, entidade.label_)

Maria PER
Paris LOC


In [ ]:
# Visualização das entidades na frase
spacy.displacy.render(texto, style='ent', jupyter=True)

In [ ]:
# O que é cada tipo de entidade(ver na doc do spacy)
print("PER:", spacy.explain('PER'))
print("LOC:", spacy.explain('LOC'))
print("ORG:", spacy.explain('ORG'))
print("MISC:", spacy.explain('MISC'))

PER: Named person or family.
LOC: Non-GPE locations, mountain ranges, bodies of water
ORG: Companies, agencies, institutions, etc.
MISC: Miscellaneous entities, e.g. events, nationalities, products or works of art


In [ ]:
text = nlp("Juliana nasceu no dia 18/06/2018 em Fortaleza")
spacy.displacy.render(text, style='ent', jupyter=True)

# 3- Criando um novo tipo de entidade no SpaCy
Aqui o modelo não identificou a data.

O spaCy permite re-treinar o modelo com novos dados melhorar o modelo e até gerar modelos novos.

Como a entidade data não foi identificada, podemos inserir alguns exemplos e re-treinar o algoritmo para que este aprenda novos padrões.

In [ ]:
import random

In [ ]:
TRAIN_DATA = [
    ("Almira é uma ótima pessoa, gosto muito dela", {"entities": [(0, 6, "PER")]}),
    ("João foi para Bahia nas férias", {"entities": [(14, 19, "LOC"), (0, 4, "PER")]}),
    ("Carlos foi visitar João na casa de praia", {"entities": [(0, 6, "PER"), (19, 23, "PER")]}),
    ("No meio do ano irei para São Paulo fazer mais um curso", {"entities": [(25, 34, "LOC")]}),
    ("O sonho dela era ir para Austrália visitar seu irmão", {"entities": [(25, 34, "LOC")]}),
    ("Em 15/07/1988 nasceu essa linda criança", {"entities": [(3, 13, "DATE"), (18, 24, "LOC")]}),
    ("Data de prisão: 10/01/2018", {"entities": [(16, 26, "DATE")]}),
    ("No dia 01/02/2016 foi decretada a sentença", {"entities": [(7, 17, "DATE")]}),
    ("A data da festa foi 07/05/2018", {"entities": [(20, 30, "DATE")]}),
    ("Dia 07/06/2020 choveu pela manhã", {"entities": [(4, 14, "DATE")]}),
    ("Michael Jackson nasceu dia 29/08/1958", {"entities": [(0, 15, "PER"), ( 27, 37, "DATE")]}),
    ("A viagem foi marcada para o dia 21/03/2021", {"entities": [(32, 42, "DATE")]}),
    ("O casamento, que era dia 12/06/2020, foi adiado para o dia 20/07/2020", {"entities": [(25, 35, "DATE"), (59, 68, "DATE")]}),
    ("Dia 21/04/2020 foi feriado", {"entities": [(4, 14, "DATE")]}),
    ("A entrega foi realizada na manhã do dia 18/03/2020 ", {"entities": [(40, 50, "DATE")]}),
    ("Na noite do dia 17/05/2019 aconteceu um eclipse", {"entities": [(16, 26, "DATE")]}),
    ("Fortaleza é a capital do Ceará", {"entities": [(0, 9, "LOC"), (25, 30, "LOC")]})
]

In [ ]:
import pandas as pd
import os
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

nlp = spacy.load("en_core_web_sm")

db = DocBin()

for text, annot in tqdm(TRAIN_DATA):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

os.chdir(r'/content')
db.to_disk("./train.spacy") # salvando o objeto

100%|██████████| 17/17 [00:00<00:00, 1408.90it/s]

Skipping entity
Skipping entity


In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     57.50    0.00    0.00    0.00    0.00
167     200         21.36    786.32  100.00  100.00  100.00    1.00
367     400          0.00      0.00  100.00  100.00  100.00    1.00
567     600          0.00      0.00  100.00  100.00  100.00    1.00
767     800          0.00      0.00  100.00  100.00  100.00    1.00
967    1000          0.00      0.00  100.00  100.00  100.00    1.00
1167    1200          0.00      0.00  100.00  100.00  100.00    1.00
1367    1400          0.00      0.00  100.00  100.00  100.00   

In [ ]:
nlp1 = spacy.load(r"/content/output/model-best")
doc = nlp1("Luana nasceu em Campo Grande em 31/05/2023")

spacy.displacy.render(doc, style="ent", jupyter=True)